In [1]:
# Import our dependencies
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import sklearn as skl
from sklearn import ensemble
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
import sqlalchemy
from sqlalchemy import create_engine

# For data cleaning - regular expressions
import re

# from config.py import db_password
import psycopg2

In [ ]:
import config
from config import db_password

### Database Connection

In [ ]:
# Database engine connection
#need to call the Database in Postgres "attrition_db"

# Postgres username, password, and database name (change accordingly)
postgres_address = 'localhost'
postgres_port = '5432' 
postgres_username = 'postgres' 
postgres_password = db_password
postgres_dbname = 'attrition_db'
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=postgres_username,
                        password=postgres_password,
                        ipaddress=postgres_address,
                        port=postgres_port,
                        dbname=postgres_dbname))
# Create the connection
engine = create_engine(postgres_str)

In [ ]:
# Load the data
attrition_df = pd.read_sql_query('''SELECT * FROM attrition_data_clean;''', engine)
attrition_df_df.head()

### Read in CSV (Alternative to DB Connection String)

In [2]:
# Read in csv file - Alternate method to read in the data if we weren't using the Postgres Database
attrition_df = pd.read_csv(Path('watson_healthcare_modified.csv'))
attrition_df.head()

,EmployeeID,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,...,RelationshipSatisfaction,StandardHours,Shift,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,1313919,41,No,Travel_Rarely,1102,Cardiology,1,2,Life Sciences,1,...,1,80,0,8,0,1,6,4,0,5
1,1200302,49,No,Travel_Frequently,279,Maternity,8,1,Life Sciences,1,...,4,80,1,10,3,3,10,7,1,7
2,1060315,37,Yes,Travel_Rarely,1373,Maternity,2,2,Other,1,...,2,80,0,7,3,3,0,0,0,0
3,1272912,33,No,Travel_Frequently,1392,Maternity,3,4,Life Sciences,1,...,3,80,0,8,3,3,8,7,3,0
4,1414939,27,No,Travel_Rarely,591,Maternity,2,1,Medical,1,...,4,80,1,6,3,3,2,2,2,2


### Data Preprocessing

In [3]:
# Initial data exploration
attrition_df.describe()

,EmployeeID,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,RelationshipSatisfaction,StandardHours,Shift,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1.676000e+03,1676.000000,1676.000000,1676.000000,1676.000000,1676.0,1676.000000,1676.000000,1676.000000,1676.000000,...,1676.000000,1676.0,1676.000000,1676.000000,1676.000000,1676.000000,1676.000000,1676.000000,1676.000000,1676.000000
mean,1.456796e+06,36.866348,800.557876,9.221957,2.907518,1.0,2.714797,65.470167,2.724940,2.066826,...,2.718377,80.0,0.806086,11.338902,2.805489,2.766110,7.033413,4.264916,2.200477,4.135442
std,2.487486e+05,9.129126,401.594438,8.158118,1.025835,0.0,1.097534,20.207572,0.714121,1.113423,...,1.078162,0.0,0.855527,7.834996,1.288431,0.702369,6.098991,3.627456,3.229587,3.559662
min,1.025177e+06,18.000000,102.000000,1.000000,1.000000,1.0,1.000000,30.000000,1.000000,1.000000,...,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,1.235832e+06,30.000000,465.000000,2.000000,2.000000,1.0,2.000000,48.000000,2.000000,1.000000,...,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,1.464606e+06,36.000000,796.500000,7.000000,3.000000,1.0,3.000000,65.500000,3.000000,2.000000,...,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,1.667992e+06,43.000000,1157.000000,14.000000,4.000000,1.0,4.000000,83.000000,3.000000,3.000000,...,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,10.000000,7.000000,3.000000,7.000000
max,1.886378e+06,60.000000,1499.000000,29.000000,5.000000,1.0,4.000000,100.000000,4.000000,5.000000,...,4.000000,80.0,3.000000,40.000000,6.000000,4.000000,40.000000,18.000000,15.000000,17.000000


In [4]:
# Find null values
attrition_df.isnull().sum()

EmployeeID                  0
Age                         0
Attrition                   0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
Shift                       0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSince

In [5]:
# determine the types of data in the columns
attrition_df.dtypes

EmployeeID                   int64
Age                          int64
Attrition                   object
BusinessTravel              object
DailyRate                    int64
Department                  object
DistanceFromHome             int64
Education                    int64
EducationField              object
EmployeeCount                int64
EnvironmentSatisfaction      int64
Gender                      object
HourlyRate                   int64
JobInvolvement               int64
JobLevel                     int64
JobRole                     object
JobSatisfaction              int64
MaritalStatus               object
MonthlyIncome                int64
MonthlyRate                  int64
NumCompaniesWorked           int64
Over18                      object
OverTime                    object
PercentSalaryHike            int64
PerformanceRating            int64
RelationshipSatisfaction     int64
StandardHours                int64
Shift                        int64
TotalWorkingYears   

In [6]:
# drop unnecessary columns (EmployeeID,Over18, EmployeeCount,StandardHours, DailyRate, HourlyRate, MonthlyRate)
attrition_df.drop(["EmployeeID","Over18","EmployeeCount","StandardHours","DailyRate","HourlyRate","MonthlyRate"],axis = 1,inplace = True)
attrition_df

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,Shift,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,No,Travel_Rarely,Cardiology,1,2,Life Sciences,2,Female,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Maternity,8,1,Life Sciences,3,Male,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Maternity,2,2,Other,4,Male,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Maternity,3,4,Life Sciences,4,Female,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Maternity,2,1,Medical,1,Male,3,...,3,4,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1671,26,Yes,Travel_Rarely,Neurology,24,3,Technical Degree,3,Male,1,...,3,2,0,1,3,1,1,0,0,0
1672,46,No,Travel_Rarely,Cardiology,10,3,Marketing,3,Female,2,...,3,3,1,15,3,3,3,2,1,2
1673,20,No,Travel_Rarely,Maternity,1,3,Life Sciences,4,Female,2,...,3,4,0,1,0,4,1,0,0,0
1674,39,No,Travel_Rarely,Neurology,1,1,Life Sciences,4,Female,2,...,3,3,1,21,3,3,21,6,11,8


### Feature Engineering

In [7]:
# Binary encoding using Pandas (multiple columns)
binary_encoded = pd.get_dummies(attrition_df, columns=["BusinessTravel", "Department","EducationField","Gender","JobRole","MaritalStatus","OverTime",])
binary_encoded.head()


,Age,Attrition,DistanceFromHome,Education,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,NumCompaniesWorked,...,JobRole_Admin,JobRole_Administrative,JobRole_Nurse,JobRole_Other,JobRole_Therapist,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes
0,41,No,1,2,2,3,2,4,5993,8,...,0,0,1,0,0,0,0,1,0,1
1,49,No,8,1,3,2,2,2,5130,1,...,0,0,0,1,0,0,1,0,1,0
2,37,Yes,2,2,4,2,1,3,2090,6,...,0,0,1,0,0,0,0,1,0,1
3,33,No,3,4,4,3,1,3,2909,1,...,0,0,0,1,0,0,1,0,0,1
4,27,No,2,1,1,3,1,2,3468,9,...,0,0,1,0,0,0,1,0,1,0


In [8]:
binary_encoded.dtypes

Age                                  int64
Attrition                           object
DistanceFromHome                     int64
Education                            int64
EnvironmentSatisfaction              int64
JobInvolvement                       int64
JobLevel                             int64
JobSatisfaction                      int64
MonthlyIncome                        int64
NumCompaniesWorked                   int64
PercentSalaryHike                    int64
PerformanceRating                    int64
RelationshipSatisfaction             int64
Shift                                int64
TotalWorkingYears                    int64
TrainingTimesLastYear                int64
WorkLifeBalance                      int64
YearsAtCompany                       int64
YearsInCurrentRole                   int64
YearsSinceLastPromotion              int64
YearsWithCurrManager                 int64
BusinessTravel_Non-Travel            uint8
BusinessTravel_Travel_Frequently     uint8
BusinessTra

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
encoded_df = binary_encoded.copy()
encoded_df["Attrition"] = le.fit_transform(encoded_df["Attrition"]) 
encoded_df.head()

,Age,Attrition,DistanceFromHome,Education,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,NumCompaniesWorked,...,JobRole_Admin,JobRole_Administrative,JobRole_Nurse,JobRole_Other,JobRole_Therapist,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes
0,41,0,1,2,2,3,2,4,5993,8,...,0,0,1,0,0,0,0,1,0,1
1,49,0,8,1,3,2,2,2,5130,1,...,0,0,0,1,0,0,1,0,1,0
2,37,1,2,2,4,2,1,3,2090,6,...,0,0,1,0,0,0,0,1,0,1
3,33,0,3,4,4,3,1,3,2909,1,...,0,0,0,1,0,0,1,0,0,1
4,27,0,2,1,1,3,1,2,3468,9,...,0,0,1,0,0,0,1,0,1,0


In [10]:
# Separate the Features (X) from the target (y)
y = encoded_df["Attrition"]
X = encoded_df.drop(columns="Attrition")

In [11]:
# Split our data into Training and Testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(1257, 44)

In [12]:
# implement random oversampling
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)


Counter({0: 1108, 1: 1108})

In [13]:
# Scale continuous data

# Creating StandardScaler instance
scaler = StandardScaler()

# Fit the Standard Scaler
X_scaler = scaler.fit(X_resampled) #not X_train

# Scale/transform the data
X_train_scaled = X_scaler.transform(X_resampled) #not X_train
X_test_scaled = X_scaler.transform(X_test)

print(X_train_scaled)

[[ 0.20496579  0.64403841  1.10393542 ...  1.16401957  0.9386983
  -0.9386983 ]
 [-0.54419286 -0.1820178   1.10393542 ... -0.85909209  0.9386983
  -0.9386983 ]
 [-0.11610221 -0.77205795  1.10393542 ... -0.85909209  0.9386983
  -0.9386983 ]
 ...
 [-0.33014753  0.64403841  1.10393542 ... -0.85909209  0.9386983
  -0.9386983 ]
 [-0.54419286  0.29001432  0.11191559 ...  1.16401957  0.9386983
  -0.9386983 ]
 [ 2.02435109  1.58810264  1.10393542 ...  1.16401957 -1.06530501
   1.06530501]]


### Create a Logistic Regression Model

In [14]:
# Create a logistic regression model
# Update the Logistic Regression model by increasing the # of estimators (100 >1000)
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='sag',
                                max_iter=1000,
                                random_state=1)
# Original solver='lbfgs'

In [15]:
# Train the model using the training data
classifier.fit(X_train_scaled, y_resampled)

LogisticRegression(max_iter=1000, random_state=1, solver='sag')

In [16]:
# Make predictions
y_pred = classifier.predict(X_test_scaled)

results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
results.head(20)

,Prediction,Actual
0,0,0
1,1,1
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,1,1
8,1,1
9,0,0


In [17]:
# Print the accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.9116945107398569


In [18]:
from sklearn.metrics import confusion_matrix, classification_report

In [19]:
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[336  33]
 [  4  46]]


In [20]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.99      0.91      0.95       369
           1       0.58      0.92      0.71        50

    accuracy                           0.91       419
   macro avg       0.79      0.92      0.83       419
weighted avg       0.94      0.91      0.92       419



### Create a Random Forest Classifier Model

In [21]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=500, random_state=78) 

In [22]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_resampled) # not y_train

In [23]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,

In [24]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,358,11
Actual 1,21,29


In [25]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

acc_score

0.9236276849642004

In [26]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,358,11
Actual 1,21,29


Accuracy Score : 0.9236276849642004
Classification Report
              precision    recall  f1-score   support

           0       0.94      0.97      0.96       369
           1       0.72      0.58      0.64        50

    accuracy                           0.92       419
   macro avg       0.83      0.78      0.80       419
weighted avg       0.92      0.92      0.92       419



### Feature Selection

In [27]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([8.94983298e-02, 5.09906872e-02, 1.68035868e-02, 2.31200679e-02,
       1.92949308e-02, 4.46964926e-02, 2.94955730e-02, 6.65399416e-02,
       2.90005043e-02, 2.16781661e-02, 3.07818388e-03, 1.45523462e-02,
       3.37743744e-02, 6.98559743e-02, 1.70989679e-02, 1.95230624e-02,
       4.88844727e-02, 3.39499441e-02, 1.84185875e-02, 3.68646523e-02,
       2.57201960e-03, 7.63025595e-03, 5.77138224e-03, 1.01972806e-02,
       7.17759206e-03, 5.72542103e-03, 1.26266697e-03, 6.00425623e-03,
       2.90191608e-03, 6.24912690e-03, 2.32844762e-03, 3.02179431e-03,
       5.67704864e-03, 6.10074838e-03, 9.86926172e-05, 1.99938132e-03,
       9.82161513e-03, 6.89566132e-03, 6.36698927e-03, 1.04148980e-02,
       7.28414795e-03, 2.14789239e-02, 9.55334136e-02, 8.03674743e-02])

In [28]:
# Sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.09553341358951906, 'OverTime_No'),
 (0.08949832976173891, 'Age'),
 (0.0803674743204394, 'OverTime_Yes'),
 (0.06985597434425084, 'TotalWorkingYears'),
 (0.06653994160070514, 'MonthlyIncome'),
 (0.05099068719145941, 'DistanceFromHome'),
 (0.04888447269560485, 'YearsAtCompany'),
 (0.044696492635380135, 'JobLevel'),
 (0.03686465227151151, 'YearsWithCurrManager'),
 (0.03394994411463568, 'YearsInCurrentRole'),
 (0.03377437437636313, 'Shift'),
 (0.02949557304328381, 'JobSatisfaction'),
 (0.029000504310035428, 'NumCompaniesWorked'),
 (0.023120067891317344, 'EnvironmentSatisfaction'),
 (0.021678166141219122, 'PercentSalaryHike'),
 (0.021478923896820816, 'MaritalStatus_Single'),
 (0.01952306243287206, 'WorkLifeBalance'),
 (0.019294930833226913, 'JobInvolvement'),
 (0.01841858750443595, 'YearsSinceLastPromotion'),
 (0.017098967871709177, 'TrainingTimesLastYear'),
 (0.016803586846078893, 'Education'),
 (0.014552346171043767, 'RelationshipSatisfaction'),
 (0.010414898016677889, 'MaritalStatus_D